In [2]:
import pandas as pd

In [7]:
labels_description = {}

df = pd.read_csv("./(V1.7)all_products_data_set.csv")
for row in range(df.shape[0]):
    label_2 = df.loc[row]["label_2nd"]
    production_description = df.loc[row]["raw_text"]
    if pd.isna(label_2) or pd.isna(production_description):
        continue
    if label_2 not in labels_description.keys():
        labels_description.update({label_2:set()})
    if not pd.isna(production_description) and production_description.__contains__("match"):
        
        labels_description.get(label_2).add(production_description)


In [10]:
count_known = 0
count_known_keys = []
for key in labels_description.keys():
    if len(labels_description.get(key)) > 0:
        count_known += 1
        count_known_keys.append(key)


Swimwear 89
Jewelry 9
Others 8
Homeware 13
Headwear 10
Pants/Trousers 18
Bag 11
Dresses 6
Lingerie 6
Short 14
Coats/Jackets 5
Skirts 3
Belt 1
Sandal 1
Glasses 11
Skin Care 5
Sweaters 9
Shirts 27
Scarf 6
Sock 7
Tank Tops 7
Leggings 2
Activewear 5
Glove 1
Scrunchie 7
Wallet 8
Watch 2
Skateboard 9
28


In [2]:
df = pd.read_csv("./test_output.csv")
df.columns

Index(['index', 'id', 'title', 'tags', 'images', 'gender', 'product_type',
       'product_type(modified)', 'main_category', 'sub_category',
       'match_most_similar_>80%_string', 'match_most_similar_>60%_string',
       'label_1st', 'label_2nd', 'label_3rd', 'buckets_num', 'buckets',
       'pattern_info', 'color_num', 'color_info', 'colors', 'url', 'raw_text'],
      dtype='object')

In [20]:
from pandarallel import pandarallel
pandarallel.initialize()

def assign_label3(str):
    if pd.isna(str):
        return ""
    else:
        str = " ".join([s.capitalize() for s in str.split(" ")])
        return str
df['label_3rd'] = df['match_most_similar_>80%_string'].parallel_apply(assign_label3)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
import json
from os import write
f = open("./json_files/colorsname_to_hex.json")
p = json.load(f)
f2 = open("my_file", "w+")
for k in p.keys():
    f2.write(k+"\n")

In [7]:
new_df = pd.read_csv("./(V1.7)all_products_data_set.csv")
new_df.columns

Index(['index', 'id', 'title', 'tags', 'images', 'gender', 'product_type',
       'product_type(modified)', 'main_category', 'sub_category',
       'match_most_similar_>80%_string', 'match_most_similar_>60%_string',
       'label_1st', 'label_2nd', 'label_3rd', 'buckets_num', 'buckets',
       'pattern_info', 'color_num', 'color_info', 'colors', 'url', 'raw_text'],
      dtype='object')

In [8]:
df = pd.read_csv("./labelled_genders.csv")
print(df.shape)
print(df.columns)


(28426, 6)
Index(['index', 'id', 'title', 'gender', 'product_type', 'buckets_num'], dtype='object')


In [20]:
type(df.loc[0]["title"])

str

In [11]:
import re
import unicodedata
not_correct = {}
id_not_correct = {}
for row in range(df.shape[0]):
    index = df.loc[row]["index"]
    id = df.loc[row]["id"]
    title = str(df.loc[row]["title"])
    
    title = re.sub("\xa0", "", title)
    if not unicodedata.is_normalized("NFKD", title):
        print(id, "not normalized")
        title = unicodedata.normalize("NFKD", title)
    
    labelled_gender = df.loc[row]["gender"]
    labelled_buckets_num = df.loc[row]["buckets_num"]
    if (new_df.loc[new_df.index == index].id).item() == id:
        if (new_df.loc[new_df.id == id].title).item() != title:
            not_correct.update({index:("title wrong", id, title)})
        new_df.loc[index, "buckets_num"] = labelled_buckets_num
    else:
        id_not_correct.update({index:("id wrong", row,id)})
print(not_correct)

6874972323922.0 not normalized
6859018666183.0 not normalized
7836190540008.0 not normalized
3732447461460.0 not normalized
3732447723604.0 not normalized
2391028760660.0 not normalized
7247205367977.0 not normalized
7247205269673.0 not normalized
4099303407700.0 not normalized
7247206449321.0 not normalized
7247206416553.0 not normalized
7247205662889.0 not normalized
7247205630121.0 not normalized
7247205531817.0 not normalized
6544829972649.0 not normalized
6544830038185.0 not normalized
4543599706196.0 not normalized
7182138474695.0 not normalized
6856818950343.0 not normalized
6856815640775.0 not normalized
6856754823367.0 not normalized
7182135099591.0 not normalized
7182130053319.0 not normalized
7182066221255.0 not normalized
6914256109767.0 not normalized
7380569456839.0 not normalized
7328771113159.0 not normalized
7318459252935.0 not normalized
7318414622919.0 not normalized
7312693395655.0 not normalized
7312694444231.0 not normalized
7308536742087.0 not normalized
72965826

In [12]:
for k, v in not_correct.items():
    if "id wrong" in v:
        print(v)
print(len(id_not_correct))

0


In [30]:
str(df.loc[31]["gender"]).replace

'women'

In [32]:
(new_df[new_df.index == 461].id)

461    4960593772626
Name: id, dtype: int64

In [72]:
stat = {}
for row in range(new_df.shape[0]):
    gender = new_df.loc[row]["gender"]
    gender = gender.replace("\xa0", "")
    if gender not in stat.keys():
        stat.update({gender: 1})
    else:
        temp = stat.get(gender)
        temp += 1
        stat.update({gender: (temp)})
stat

{'women': 15371,
 'na': 2043,
 'unisex': 2403,
 'broken': 477,
 'men': 6156,
 'unknown': 43,
 'girls': 224,
 'unset': 591,
 'boys': 288,
 'unisex/unknown': 830}

In [73]:
total = new_df.shape[0]
for item in stat.keys():
    temp = stat.get(item)
    new_data = round ((temp * 100 / total), 2)
    stat.update({item: (str(new_data) + "%")})
stat


{'women': '54.07%',
 'na': '7.19%',
 'unisex': '8.45%',
 'broken': '1.68%',
 'men': '21.66%',
 'unknown': '0.15%',
 'girls': '0.79%',
 'unset': '2.08%',
 'boys': '1.01%',
 'unisex/unknown': '2.92%'}

In [54]:
new_df.to_csv("(V1.8)all_products_data_set.csv", index=False)